In [2]:
from __future__ import print_function
import torch
import argparse
import sys
sys.path.append("../")
import platform
from datetime import timedelta, date, datetime
import numpy as np
import os
import time
import util.data_processing_tool as dpt
from netCDF4 import Dataset
import cv2
import tqdm

E:\Users\Weifa\Anaconda3\envs\py37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
E:\Users\Weifa\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
E:\Users\Weifa\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
E:\Users\Weifa\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [4]:
def list_all_files(rootdir):
    import os
    _files = []
    list = os.listdir(rootdir) #列出文件夹下所有的目录与文件
    for i in range(0,len(list)):
        path = os.path.join(rootdir,list[i])
        if os.path.isfile(path):
            _files.append(path)
    return _files

class gridboxavg:
    def __init__(self, kernel_size=(2, 2), stride=2):
        self.stride = stride
        self.kernel_size = kernel_size
        self.w_height = kernel_size[0]
        self.w_width = kernel_size[1]

    def __call__(self, x):
        self.x = x
        self.in_height = x.shape[0]
        self.in_width = x.shape[1]

        self.out_height = int((self.in_height - self.w_height) / self.stride) + 1
        self.out_width = int((self.in_width - self.w_width) / self.stride) + 1
        out = np.zeros((self.out_height, self.out_width))

        for i in range(self.out_height):
            for j in range(self.out_width):
                start_i = i * self.stride
                start_j = j * self.stride
                end_i = start_i + self.w_height
                end_j = start_j + self.w_width
                out[i, j] = np.mean(x[start_i: end_i, start_j: end_j])
        return out

In [11]:
    barra_file_list=list_all_files('../../Data/barra_aus/')
    # a=[i.split('/')[-1] for i in a]
    grid_box_avg = gridboxavg((3, 3), stride=3)
    barra_file_list=tqdm.tqdm(barra_file_list)
    for i in barra_file_list:
        file_name=i.split('/')[-1]
        data=Dataset(i,'r')

        var=data['barra'][:]
        var=grid_box_avg(var)
        var=dpt.interp_tensor_2d(var,(79,94),interp= cv2.INTER_LINEAR)
    #     print(var.shape)
        var,lat,lon=dpt.add_lat_lon_data(var)
        data.close()
        dpt.mkdir('../data/barra_aus/barra_upscaled/')

        f_w = Dataset('../data/barra_aus/barra_upscaled/'+file_name,'w',format = 'NETCDF4')
        f_w.createDimension('lat',len(lat))
        f_w.createDimension('lon',len(lon))

        f_w.createVariable('lat',np.float32,('lat'))
        f_w.createVariable('lon',np.float32,('lon'))

        f_w.variables['lat'][:] = lat
        f_w.variables['lon'][:] = lon

        f_w.createVariable( 'barra', np.float32, ('lat','lon'))
        f_w.variables['barra'][:] =var
        f_w.close()


100%|███████████████████████████████████████████████████████████████████████████▉| 8759/8760 [2:46:23<00:01,  1.17s/it]

OSError: [Errno -101] NetCDF: HDF error: b'../../Data/barra_aus/barra_aus.zip'

In [37]:
#calculate the scale factor


barra_file_list=list_all_files('../../Data/barra_aus/')
barra_upscale_file_list=list_all_files('../../Data/barra_aus/barra_upscaled/')

date_list=dpt.date_range(date(2012,1,1),date(2012,12,31))
# .timetuple().tm_yday

dayofyear=[]
for t in date_list:
#     print(t)

    origin=[]
    upscale=[]
    for y in range(1990,2013):
        try:
            taget=date(y,t.month,t.day)
        except:
            continue
            
        
        file_name='../../Data/barra_aus/'+(taget+timedelta(1)).strftime('%Y%m%d')+'.nc'
        
        if os.path.exists(file_name):
            origin.append(dpt.read_barra_data_fc('../../Data/barra_aus/',taget))
            upscale.append(dpt.interp_tensor_2d(dpt.read_barra_data_fc('../../Data/barra_aus/barra_upscaled/',taget),(316,376),interp=cv2.INTER_LINEAR) )
        else:
            print(file_name)
    origin=np.array(origin).mean(axis=0)
    upscale=np.array(upscale).mean(axis=0)
    print
    
    dayofyear.append(origin/upscale)

np.save('scale_factor.npy',np.array(dayofyear))






# for i in 
#         for file in file_list:

#             ensemble_mem=[]
# #             year=[]
#             historical_year=file.split('/')[-1][6:10]
#             historical_month=file.split('/')[-1][10:12]
#             historical_day=file.split('/')[-1][12:14] 
# for i,j in zip(barra_file_list,barra_upscale_file_list):
    
#     break

In [38]:
np.load('scale_factor.npy').shape

(366, 316, 376)

In [20]:

date(1997,2,13).strftime('%Y%m%d')
# .timetuple().tm_yday

'19970213'